# Retrieval-augmented generation with open-source, quantized large language models

## Introduction

Retrieval-augmented generation (RAG) is a technique that combines the benefits of large-scale pre-trained language models (LMs) with the benefits of retrieving relevant information from a large corpus. We will use this framework to build a chat system that will be able to answer questions about a given context based on selective data.

The technique is described in the paper [Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks](https://arxiv.org/abs/2005.11401).

## Setup

First, we need to import some libraries. [PyTorch](https://pytorch.org/) is a library for tensor computation and deep learning. We will use the torch utilities to interact with our GPU. [Transformers](https://huggingface.co/docs/transformers/index) is a library for state-of-the-art NLP models. We will use it to load our model and tokenizer. [Sentence Transformers](https://www.sbert.net/) is a library for state-of-the-art sentence embeddings. We will use it to encode our context and queries. [Faiss](https://faiss.ai) is a library for efficient similarity search. We will use it to retrieve relevant information from our corpus.

In [1]:
import torch
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

For educational reason we will use here an unquantized model as a start point. Quantization is a technique to reduce the memory footprint of a model tremendously. This is achieved by reducing the precision of the model's weights. It allows us to use a larger model than we would normally be able to fit on a GPU. Further details about the technique are described in the papers [Quantization and Training of Neural Networks for Efficient Integer-Arithmetic-Only Inference](https://arxiv.org/abs/1712.05877) and [AWQ: Activation-aware Weight Quantization for LLM Compression and Acceleration](https://arxiv.org/abs/2306.00978). The download will take some time hence we surpress the execution here and only provide the outcome.

In [2]:
# !git lfs clone https://huggingface.co/mistralai/Mistral-7B-v0.1 ../resource/model/Mistral-7B-v0.1
# WARNING: 'git lfs clone' is deprecated and will not be updated
#           with new flags from 'git clone'

# 'git clone' has been updated in upstream Git to have comparable
# speeds to 'git lfs clone'.
# Cloning into '../resource/model/Mistral-7B-v0.1'...
# remote: Enumerating objects: 79, done.
# remote: Counting objects: 100% (75/75), done.
# remote: Compressing objects: 100% (74/74), done.
# remote: Total 79 (delta 39), reused 0 (delta 0), pack-reused 4
# Unpacking objects: 100% (79/79), 470.69 KiB | 2.39 MiB/s, done.
# Downloading LFS objects: 100% (3/3), 15 GB | 32 MB/s  

In [3]:
!tree -L 1 ../resource/model/Mistral-7B-v0.1

../resource/model/Mistral-7B-v0.1
├── README.md
├── config.json
├── generation_config.json
├── pytorch_model-00001-of-00002.bin
├── pytorch_model-00002-of-00002.bin
├── pytorch_model.bin.index.json
├── special_tokens_map.json
├── tokenizer.json
├── tokenizer.model
└── tokenizer_config.json

0 directories, 10 files


In [4]:
!cat ../resource/model/Mistral-7B-v0.1/config.json

{
  "architectures": [
    "MistralForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.34.0.dev0",
  "use_cache": true,
  "vocab_size": 32000
}


In [5]:
!du -sh ../resource/model/Mistral-7B-v0.1

28G	../resource/model/Mistral-7B-v0.1


In [6]:
!nvidia-smi --query-gpu=gpu_name,memory.total --format=csv

name, memory.total [MiB]
NVIDIA GeForce RTX 3060, 12288 MiB


Now we are loading the default [model configuration](https://huggingface.co/docs/transformers/main_classes/configuration) from our local model base. The name 'model_name_or_path' already indicates, that we also could use a model directly from the [Hugging Face model hub](https://huggingface.co/models) and download it on demand. This is handy in situation where if we would like to check different models and it ensures that we always use the latest version of the model. Anyhow, for this tutorial we will use the model code we already have downloaded.

In [7]:
model_name_or_path = "../resource/model/Mistral-7B-v0.1"

# Load model config
model_config = transformers.AutoConfig.from_pretrained(
    model_name_or_path,
)

Next we set some preconfigurations for our model loading with the help of the library [bitsandbytes](https://github.com/TimDettmers/bitsandbytes). We are using the enhancement of shrinking the model weights to 4-bit integers. This will reduce the memory footprint of our model roughly by a factor of 4! The technique is described in the blog post [Making LLMs even more accessible with bitsandbytes, 4-bit quantization and QLoRA](https://huggingface.co/blog/4bit-transformers-bitsandbytes).

In [8]:
# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Retrieving the availabile compute dtype from torch
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Initializing the bitsandbytes config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

Your GPU supports bfloat16: accelerate training with bf16=True


Loading our model from our local code base with our preconfigurations.

In [9]:
# Loading our model
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    quantization_config=bnb_config,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Now initializing the [tokenizer](https://huggingface.co/docs/transformers/main_classes/tokenizer). The tokenizer is used to convert our text input into a numerical representation that can be processed by the model.

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

---

## Pipeline

In [ ]:
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.8,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,
    do_sample=True,
)

In [ ]:

from langchain import LLMChain
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import AsyncChromiumLoader
from langchain.document_transformers import html2text
from langchain.schema.runnable.passthrough import RunnablePassthrough


In [ ]:
import nest_asyncio
nest_asyncio.apply()

# Articles to index
articles = ["https://www.fantasypros.com/2023/11/rival-fantasy-nfl-week-10/",
            "https://www.fantasypros.com/2023/11/5-stats-to-know-before-setting-your-fantasy-lineup-week-10/",
            "https://www.fantasypros.com/2023/11/nfl-week-10-sleeper-picks-player-predictions-2023/",
            "https://www.fantasypros.com/2023/11/nfl-dfs-week-10-stacking-advice-picks-2023-fantasy-football/",
            "https://www.fantasypros.com/2023/11/players-to-buy-low-sell-high-trade-advice-2023-fantasy-football/"]

# Scrapes the blogs above
loader = AsyncChromiumLoader(articles)
docs = loader.load()

In [ ]:
# Converts HTML to plain text 
html2text = html2text.Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

In [ ]:
# Chunk text
text_splitter = CharacterTextSplitter(chunk_size=100, 
                                      chunk_overlap=0)
chunked_documents = text_splitter.split_documents(docs_transformed)

In [ ]:
# Load chunked documents into the FAISS index
db = FAISS.from_documents(chunked_documents, 
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

# Prepare the db to retrieve input
retriever = db.as_retriever()

In [ ]:
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

prompt_template = """
### [INST] 
Instruction: Answer the question based on your 
aviation knowledge. Here is context to help:

{context}

### QUESTION:
{question} 

[/INST]
 """

# Create prompt from prompt template 
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain 
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

In [ ]:
rag_chain = ( 
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

rag_chain.invoke("Should I start Gibbs next week for fantasy?")